In [2]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [3]:
# Importing self-made libaries
import sensitivity as st
import Monte_Carlo as mc
import reload_lib as rl
import life_cycle_assessment as lc
import sens_table as tab
import LCA_plots as lp
import sensitivity_case1 as c1
import sensitivity_case2 as c2


lib = [st, mc, lc, tab, lp, c1, c2]

In [16]:
from copy import deepcopy as dc

In [4]:
# Path to where the code is stored
path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care'
# Specifying the LCIA method
lcia_method = 'recipe'

ecoinevnt_paths = {'ev391apos' : r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets",
                   'ev391consq' : r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets",
                   'ev391cutoff' : r"C:\Users\ruw\Downloads\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets"}
system_path = [r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case1.xlsx', 
                r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case2.xlsx']

In [ ]:
rl.reload_lib(lib)
variables = st.break_even_initialization(path, lcia_method)

case1_apos already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_apos
case1_consq already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_consq
case1_cut_off already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_cut_off
case2_apos already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case2_apos
case2_consq already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case2_consq
case2_cut_off already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case2_cut_off
Recipe is selected


In [419]:
database_name, df_GWP, db_type, save_dir, impact_category, flows = variables['case1_apos']
if 'case1' in database_name:
        flow_legend = [
                    'H2S',
                    'H2R',
                    'ASC',
                    'ASW',
                    'H4S',
                    'H4R',
                    'ALC',
                    'ALW'
                    ]

else:
    flow_legend = ['SUD', 'MUD']

# Creating the dataframe for min and max values
columns = lc.unique_elements_list(database_name)

df_stack_updated, totals_df = lp.process_categorizing(df_GWP, db_type, database_name, 'break even', flow_legend, columns)
# Calling the function to have the different activiteis split into the correct column in the dataframe
df_be = lp.break_even_orginization(df_stack_updated, database_name)
df_be

,Raw mat. + prod.,Disinfection,Autoclave,Recycling,Incineration,Avoided energy prod.,Avoided mat. prod.
H2S,0.247668,0.0,0.222591,0.015494,0.203083,-0.123138,0.0
H2R,0.247668,0.0,0.222591,0.067563,0.060219,-0.018471,-0.146522
ASC,0.079466,0.11211,0.333886,0.007174,0.00075,-0.000155,-0.026781
ASW,0.079466,0.117369,0.333886,0.007174,0.00075,-0.000155,-0.026781
H4S,0.708736,0.0,0.296788,0.04408,0.576358,-0.352291,0.0
H4R,0.708736,0.0,0.296788,0.191265,0.087234,-0.052844,-0.412934
ALC,0.133437,0.22422,0.445182,0.006792,0.000375,-0.00031,-0.04478
ALW,0.133437,0.073726,0.445182,0.006792,0.000375,-0.00031,-0.04478


In [420]:
df, val_dct, idx_sens, col_to_df = c1.case1_initilazation(df_be)
val_dct

{'Life time': {'ASC': [314, 827],
  'ASW': [314, 827],
  'ALC': [314, 827],
  'ALW': [314, 827]},
 'autoclave': {'H2S': [12, 18],
  'H2R': [12, 18],
  'ASC': [9, 12],
  'ASW': [9, 12],
  'H4S': [8, 9],
  'H4R': [8, 9],
  'ALC': [6, 9],
  'ALW': [6, 9]},
 'protection cover': {'H2S': [0.063, 0.071],
  'H2R': [0.063, 0.071],
  'H4S': [0.19, 0.202],
  'H4R': [0.19, 0.202]}}

In [375]:
def calculate_autoclave_temp(col, df_be, i, c, val):
    if '2' in col:
        return df_be.at[i, c] * val / 12
    elif '4' in col:
        return df_be.at[i, c] * val / 8
    elif 'AS' in col:
        return df_be.at[i, c] * val / 9
    elif 'AL' in col:
        return df_be.at[i, c] * val / 6
    else:
        return 0

In [422]:
rl.reload_lib(lib)

df, val_dct, idx_sens, col_to_df = c1.case1_initilazation(df_be)
df_dct = {}
for cr in df.columns:
        df_dct[cr] = {}
        for ir, rr in df.iterrows():
                # print(ir)
                temp = 0
                df_sens = dc(df_be)
                # print(cr, ir)
                # dct_idx = cr + ' - ' + ir
                
                for col in df_sens.columns:
                        # print(col)
                        for idx, row in df_sens.iterrows():
                                if ir != 'total':
                                        dct = val_dct[ir]
                                        val = 0 if 'lower' in cr else 1

                                        if ir == 'Life time' and idx in cr  and 'H' not in idx and 'Disinfection' not in col and 'Autoclave' not in col:
                                                row[col] *=  513 / dct[idx][val] 

                                        elif ir == 'autoclave' and 'Autoclave' in col and idx in cr:
                                                # print(cr, ir, col)
                                                if '2' in cr:
                                                        row[col] *= 12 / dct[idx][val]
                                                elif '4' in cr:
                                                        
                                                        row[col] *= 8 /dct[idx][val]
                                                elif 'AS' in cr:
                                                        row[col] *= 9/ dct[idx][val]
                                                elif 'AL' in cr:
                                                        row[col] *=  6 / dct[idx][val]
                                        elif ir == 'protection cover' and idx in cr and 'A' not in idx and 'Disinfection' not in col and 'Autoclave' not in col and 'Recycling' not in col :
                                                # print('prot', idx, cr, col, row[col], dct[idx][val])
                                               row[col] *= dct[idx][val] / dct[idx][1]
                # print(cr, ir)
                df_temp = df_sens.loc[cr[:3]].to_frame().T
                # print(cr[:3], ir, df_temp)
                df_dct[cr].update({ir : df_temp})


In [423]:
df_perc = dc(df)
for sc, case in df_dct.items():
    # print(sc, case)
    for desc, data in case.items():
        if desc != 'total':
            
            idx = [i for i in data.index][0]

            lst1 = data.loc[idx].to_list()
            lst2 = df_be.loc[idx].to_list()
            tot_sens = 0
            tot = 0
            for k, n in enumerate(lst1):
                tot_sens += n
                tot += lst2[k]
            # print(idx, desc, tot, tot_sens)
            p = (tot_sens - tot)/tot
            df_perc.at[desc, sc] = p

df_perc

,H2S - lower%,H2S - upper%,H2R - lower%,H2R - upper%,ASC - lower%,ASC - upper%,ASW - lower%,ASW - upper%,H4S - lower%,H4S - upper%,H4R - lower%,H4R - upper%,ALC - lower%,ALC - upper%,ALW - lower%,ALW - upper%
Life time,0.0,0.0,0.0,0.0,0.075651,-0.045322,0.074873,-0.044857,0.0,0.0,0.0,0.0,0.079136,-0.047411,0.098519,-0.059023
autoclave,0.0,-0.13116,0.0,-0.171336,0.0,-0.164817,0.0,-0.163123,0.0,-0.025891,0.0,-0.040301,0.0,-0.194,0.0,-0.241518
protection cover,-0.065254,0.0,-0.03718,0.0,0.0,0.0,0.0,0.0,-0.043507,0.0,-0.023973,0.0,0.0,0.0,0.0,0.0
total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [424]:
df_perc_tot = dc(df_perc)
for col in df_perc_tot.columns:
    for idx, row in df_perc_tot.iterrows():
        if 'total' not in idx:
            df_perc_tot.at['total', col] += row[col]
        if row[col] == 0:
            row[col] = '-'

df_perc_tot

,H2S - lower%,H2S - upper%,H2R - lower%,H2R - upper%,ASC - lower%,ASC - upper%,ASW - lower%,ASW - upper%,H4S - lower%,H4S - upper%,H4R - lower%,H4R - upper%,ALC - lower%,ALC - upper%,ALW - lower%,ALW - upper%
Life time,-,-,-,-,0.075651,-0.045322,0.074873,-0.044857,-,-,-,-,0.079136,-0.047411,0.098519,-0.059023
autoclave,-,-0.13116,-,-0.171336,-,-0.164817,-,-0.163123,-,-0.025891,-,-0.040301,-,-0.194,-,-0.241518
protection cover,-0.065254,-,-0.03718,-,-,-,-,-,-0.043507,-,-0.023973,-,-,-,-,-
total,-0.065254,-0.13116,-0.03718,-0.171336,0.075651,-0.210139,0.074873,-0.20798,-0.043507,-0.025891,-0.023973,-0.040301,0.079136,-0.241411,0.098519,-0.300542


In [407]:
import pandas as pd

In [468]:
database_name, df_GWP, db_type, save_dir, impact_category, flows = variables['case2_apos']
if 'case1' in database_name:
        flow_legend = [
                    'H2S',
                    'H2R',
                    'ASC',
                    'ASW',
                    'H4S',
                    'H4R',
                    'ALC',
                    'ALW'
                    ]

else:
    flow_legend = ['SUD', 'MUD']

# Creating the dataframe for min and max values
columns = lc.unique_elements_list(database_name)

df_stack_updated, totals_df = lp.process_categorizing(df_GWP, db_type, database_name, 'break even', flow_legend, columns)
# Calling the function to have the different activiteis split into the correct column in the dataframe
df_be = lp.break_even_orginization(df_stack_updated, database_name)
autoclave_gwp = tab.autoclave_gwp_impact_case1(variables['case1_apos'], path)
df, val_dct, idx_sens, col_to_df = c2.case2_initilazation(df_be, db_type, autoclave_gwp)
autoclave_gwp = tab.autoclave_gwp_impact_case1(variables['case1_apos'], path)

use_elec = ((60-4)/60*40 + 500 * 4/60)/1000

idx_col = totals_df.columns[0]
idx_tot = [i[0] for i in totals_df[idx_col]]
totals_df.index = idx_tot

'H200 SU' (unit, GLO, None) 0.0857768559144634
case1_apos already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_apos
Recipe is selected
case1_apos already exist
C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\results\case1_apos


In [483]:
val_dct

{'autoclave': {'MUD': [36, 54]},
 'surgery time': {'SUD': [0.05533333333333333, 0.11666666666666665],
  'MUD': [0.05533333333333333, 0.11666666666666665]},
 'sterilization': {'MUD': [0.028206550700787716, 0.1628137911203456]},
 'Life time': {'MUD': [50, 500]},
 'cabinet washer': {'MUD': [32, 48]}}

In [ ]:
rl.reload_lib(lib)
df, val_dct, idx_sens, col_to_df = c2.case2_initilazation(df_be, db_type, autoclave_gwp)

df_dct = {}

for cr in df.columns:
        df_dct[cr] = {}
        for ir, rr in df.iterrows():
            df_sens = dc(df_be)
            for col in df_sens.columns:
                for idx, row in df_sens.iterrows():
                    if ir != 'total':
                        dct = val_dct[ir]
                        val = 0 if 'lower' in cr else 1
                        if ir == 'Life time' and idx in cr and 'SUD' not in cr and 'Disinfection' not in col and 'autoclave' not in col:                                        
                            row[col] *= 250 / dct[idx][val]
                            
                        elif ir == 'autoclave' and 'Autoclave' in col and 'SUD' not in idx:
                            row[col] *= (12*4) / dct[idx][val]
                        elif ir == 'sterilization' and 'consumables' in col and 'SUD' not in idx:
                            row[col] = dct[idx][val]
                        elif ir == 'cabinet washer' and 'Disinfection' in col and 'SUD' not in idx:
                            row[col] *= 32 / dct[idx][val]
                        elif ir == 'surgery time':
                            row[col] *= use_elec / dct[idx][val]
            df_temp = df_sens.loc[cr[:3]].to_frame().T
            df_dct[cr].update({ir : df_temp})

df_dct['MUD - upper%']['cabinet washer']

Recipe is selected


,Raw mat. + prod.,Use,Disinfection,Ster. consumables,Ster. autoclave,Incineration,Avoided energy prod.
MUD,0.226283,0.01623,0.07474,0.085777,0.055648,0.014437,-0.073263


In [492]:
df_perc = dc(df)
for sc, case in df_dct.items():
    # print(sc, case)
    for desc, data in case.items():
        if desc != 'total':
            
            idx = [i for i in data.index][0]

            lst1 = data.loc[idx].to_list()
            lst2 = df_be.loc[idx].to_list()
            tot_sens = 0
            tot = 0
            for k, n in enumerate(lst1):
                tot_sens += n
                tot += lst2[k]
            # print(idx, desc, tot, tot_sens)
            p = (tot_sens - tot)/tot
            df_perc.at[desc, sc] = p

df_perc_tot = dc(df_perc)
for col in df_perc_tot.columns:
    for idx, row in df_perc_tot.iterrows():
        if 'total' not in idx:
            df_perc_tot.at['total', col] += row[col]
        if row[col] == 0:
            row[col] = '-'

df_perc_tot

,SUD - lower%,SUD - upper%,MUD - lower%,MUD - upper%
autoclave,-,-,-,-
cabinet washer,-,-,-,-0.085472
Life time,-,-,2.465236,-0.308155
sterilization,-,-,-0.131673,0.176197
surgery time,0.277108,-0.394286,0.277108,-0.394286
total,0.277108,-0.394286,2.610671,-0.611715


In [ ]:


# for col in df_sensitivity_v.columns:
#     for idx, row in df_sensitivity_v.iterrows():
#         for key, dct in val_dct.items():
#                 if idx in key:
#                     for i, r in df_be.iterrows():
#                         for c in df_be.columns:
#                             for sc, lst in dct.items():
#                                 temp = 0
#                                 tot_baseline = 0
#                                 if sc in i:
#                                     tot_baseline += df_be.at[i, c]
#                                     if 'lower' in col:
#                                         val = lst[0]
#                                         tidx = col.replace(" - lower%", "")
#                                     else:
#                                         val = lst[1]
#                                         tidx = col.replace(" - upper%", "")
                                    
#                                     if idx == 'Life time' and i in col and 'SUD' not in col and 'Disinfection' not in c and 'autoclave' not in c:                                        
#                                         row[col] += (df_be.at[i, c] * 250/val)
                                        
#                                     elif idx == 'autoclave' and 'autoclave' in c and 'SUD' not in col:
#                                         row[col] += (df_be.at[i, c] * (12*4) / val)
#                                     elif idx == 'sterilization' and 'consumables' in c and 'SUD' not in col:
#                                         row[col] += val
#                                     elif idx == 'cabinet washer' and 'Disinfection' in c and 'SUD' not in col:
#                                         row[col] += (df_be.at[i, c] * 32 /val)
                                        
#                                     elif idx == 'surgery time' and 'Disinfection' in c:
#                                         row[col] += (df_be.at[i, c] * use_elec / val)


df_sensitivity_v

In [ ]:

use_elec = ((60-4)/60*40 + 500 * 4/60)/1000
df_sensitivity_v = pd.DataFrame(0, index=idx_sens, columns=col_to_df, dtype=object)

idx_col = totals_df.columns[0]
idx_tot = [i[0] for i in totals_df[idx_col]]
totals_df.index = idx_tot

tot_lst = []
for tidx in totals_df.index:
    tot_lst.append(totals_df.at[tidx, 'Value'])

df_dct = {}

for cr in df.columns:
        df_dct[cr] = {}
        for ir, rr in df.iterrows():
            df_sens = dc(df_be)
            for col in df_sens.columns:
                        # print(col)
                        for idx, row in df_sens.iterrows():
                            if ir != 'total':
                                dct = val_dct[ir]
                                val = 0 if 'lower' in cr else 1
                            


for col in df_sensitivity_v.columns:
    for idx, row in df_sensitivity_v.iterrows():
        for key, dct in val_dct.items():
                if idx in key:
                    for i, r in df_be.iterrows():
                        for c in df_be.columns:
                            for sc, lst in dct.items():
                                temp = 0
                                tot_baseline = 0
                                if sc in i:
                                    tot_baseline += df_be.at[i, c]
                                    if 'lower' in col:
                                        val = lst[0]
                                        tidx = col.replace(" - lower%", "")
                                    else:
                                        val = lst[1]
                                        tidx = col.replace(" - upper%", "")
                                    
                                    if idx == 'Life time' and i in col and 'SUD' not in col and 'Disinfection' not in c and 'autoclave' not in c:                                        
                                        row[col] += (df_be.at[i, c] * 250/val)
                                        
                                    elif idx == 'autoclave' and 'autoclave' in c and 'SUD' not in col:
                                        row[col] += (df_be.at[i, c] * (12*4) / val)
                                    elif idx == 'sterilization' and 'consumables' in c and 'SUD' not in col:
                                        row[col] += val
                                    elif idx == 'cabinet washer' and 'Disinfection' in c and 'SUD' not in col:
                                        row[col] += (df_be.at[i, c] * 32 /val)
                                        
                                    elif idx == 'surgery time' and 'Disinfection' in c:
                                        row[col] += (df_be.at[i, c] * use_elec / val)


df_sensitivity_v